In [1]:
import sys
import os
import warnings
import logging
import numpy as np
import pandas as pd
import random as rn
import tracemalloc
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.client import device_lib

from tensorflow.keras import backend as K

# import helper_functions.config as config
import helper_functions.data_functions as data_functions
#import helper_functions.plotting_functions as plotting_functions

# Note: tf 2.1.0 give warning about model weight format when
# using class weights. This is the only way to silence without
# updating

def warn(*args, **kwargs):
    pass

tf.get_logger().setLevel(logging.ERROR)
warnings.warn = warn

tf.random.set_seed(42)
np.random.seed(42)
rn.seed(42)
os.environ['PYTHONHASHSEED']=str(42)

print(f"Python {sys.version}")
print()
print(f"Pandas {pd.__version__}")
print(f"Tensorflow {tf.__version__}")
print(f"Keras {keras.__version__}")
print()

devices = device_lib.list_local_devices()

if 'GPU' in ('').join(str(devices)):
    print("tf accessable GPU found: "+devices[-2].physical_device_desc)

Python 3.7.7 (default, Mar 23 2020, 22:36:06) 
[GCC 7.3.0]

Pandas 1.0.3
Tensorflow 2.2.0
Keras 2.3.0-tf

tf accessable GPU found: device: 0, name: GeForce GTX 1070, pci bus id: 0000:02:00.0, compute capability: 6.1


In [2]:
class NeuralNet:
    def __init__(self, hyperparameters, metrics, data):
        self.recurrent_unit_type = hyperparameters['recurrent_unit_type']
        self.statefullness = hyperparameters['statefullness']
        self.units = hyperparameters['units']
        self.learning_rate = hyperparameters['learning_rate']
        self.batch_size = hyperparameters['batch_size']
        self.past_history = hyperparameters['past_history']
        self.dropout = hyperparameters['dropout']
        self.class_weight = hyperparameters['class_weight']
        self.output_bias = hyperparameters['initial_output_bias']
        self.future_target = hyperparameters['future_target']
        self.step = hyperparameters['step']
        self.test_data_frac = hyperparameters['test_data_frac']
        self.validation_data_frac = hyperparameters['validation_data_frac']
        self.training_epochs = hyperparameters['training_epochs']
        self.verbose = hyperparameters['verbose']
            
        self.metrics = metrics
        
        self.data = data
        
    def prep_data(self):
        # sort dataframe by date
        self.data = self.data.sort_values(by='date')
        
        # split data up into training, testing and validation sets
        num_observations = len(self.data)
        
        test_data = data.tail(int(len(self.data) * self.test_data_frac))
        leftover_data = data.head(int(len(self.data) * (1 - self.test_data_frac)))
        
        validation_data = data.tail(int(len(leftover_data) * self.validation_data_frac))
        training_data = data.head(int(len(leftover_data) * (1 - self.validation_data_frac)))
        
        self.steps_per_epoch = int(len(training_data) // self.batch_size)
        self.validation_steps = int(len(validation_data) // self.batch_size)

        # create training and vaildation data 
        x_train, y_train = data_functions.multivariate_data(
            training_data,
            self.past_history,
            self.future_target, 
            self.step
        )

        x_validation, y_validation = data_functions.multivariate_data(
            validation_data,
            self.past_history,
            self.future_target, 
            self.step
        )
        
        x_test, y_test = data_functions.multivariate_data(
            test_data,
            self.past_history,
            self.future_target, 
            self.step
        )

        # trim datasets so they are whole number multiples of the batch
        # size (needed for statefull LSTM)
        start_index = (x_train.shape[0] - (x_train.shape[0] % self.batch_size))
        end_index = x_train.shape[0]

        self.x_train = np.delete(x_train, range(start_index, end_index), axis=0)
        self.y_train = np.delete(y_train, range(start_index, end_index), axis=0)

        start_index = (x_validation.shape[0] - (x_validation.shape[0] % self.batch_size))
        end_index = x_validation.shape[0]

        self.x_validation = np.delete(x_validation, range(start_index, end_index), axis=0)
        self.y_validation = np.delete(y_validation, range(start_index, end_index), axis=0)
        
        start_index = (x_test.shape[0] - (x_test.shape[0] % self.batch_size))
        end_index = x_test.shape[0]

        self.x_test = np.delete(x_test, range(start_index, end_index), axis=0)
        self.y_test = np.delete(y_test, range(start_index, end_index), axis=0)

        input_dim = self.x_train.shape[-2:]
        self.input_shape = (self.batch_size, input_dim[0], input_dim[1])
        
        return True
    
    def build_model(self):
        
        model = tf.keras.models.Sequential()

        if self.recurrent_unit_type == 'LSTM':
            model.add(tf.keras.layers.LSTM(
                self.units,
                batch_input_shape = self.input_shape,
                stateful = self.statefullness,
                dropout = self.dropout
            ))
            
        elif self.recurrent_unit_type == 'GRU':
            model.add(tf.keras.layers.GRU(
                self.units,
                stateful = self.statefullness,
                dropout = self.dropout
            ))
            
        model.add(tf.keras.layers.Dense(
            1,
            activation = 'sigmoid',
            bias_initializer = self.output_bias)
        )

        model.compile(
            optimizer = tf.keras.optimizers.Adam(lr = self.learning_rate), 
            loss = tf.keras.losses.BinaryCrossentropy(),
            metrics = self.metrics
        )
        
        self.model = model
        
        return True
        
    def train_model(self):
        history = self.model.fit(
            self.x_train,
            self.y_train,
            epochs = self.training_epochs,
            batch_size = self.batch_size,
            steps_per_epoch = self.steps_per_epoch,
            validation_data = (self.x_validation, self.y_validation),
            validation_steps = self.validation_steps,
            class_weight = self.class_weight,
            verbose = self.verbose
        )
        
        return history
    
    def evaluate_model(self):
        results = self.model.evaluate(
            self.x_test,
            self.y_test,
            batch_size = self.batch_size,
            verbose = self.verbose
        )
        
        return results

In [3]:
metrics = {
    'true_positives': keras.metrics.TruePositives(name='true_positives'),
    'false_positives': keras.metrics.FalsePositives(name='false_positives'),
    'true_negatives': keras.metrics.TrueNegatives(name='true_negatives'),
    'false_negatives': keras.metrics.FalseNegatives(name='false_negatives'), 
    'AUC': keras.metrics.AUC(name='AUC'),
    'matthews_correlation': data_functions.matthews_correlation,
    'F1': data_functions.f1
}

In [4]:
# Load data
data_file = '/mnt/SSD/data/training_data/1992-2015_california_standard_scaled_mean_range_onehot_month_selective_box_cox.parquet'
raw_data = pd.read_parquet(data_file)

# Pick one spatial bin with fires
data = raw_data[(raw_data['raw_lat'] == 39.42233) & (raw_data['raw_lon'] == -120.6546)]

# Also drop unnecessary columns
data.drop([
    'lat',
    'lon',
    'raw_lat',
    'raw_lon',
    'total_fires',
    'veg',
    'January',
    'February',
    'March',
    'April',
    'May',
    'June',
    'July',
    'August',
    'Septermber',
    'October',
    'November',
    'December'], axis=1, inplace=True)

data.reset_index(inplace=True, drop=True)

# calculate class weights and initial output bias from ignition frequency in the data 
class_weight = data_functions.get_class_weights(data)
output_bias = data_functions.get_initial_output_bias(data)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8766 entries, 0 to 8765
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   apcp               8766 non-null   float32       
 1   crain              8766 non-null   int32         
 2   ignition           8766 non-null   int32         
 3   mean_air_2m        8766 non-null   float32       
 4   mean_rhum_2m       8766 non-null   float32       
 5   mean_dpt_2m        8766 non-null   float32       
 6   mean_pres_sfc      8766 non-null   float32       
 7   mean_uwnd_10m      8766 non-null   float32       
 8   mean_vwnd_10m      8766 non-null   float32       
 9   mean_vis           8766 non-null   float32       
 10  mean_cloud_cover   8766 non-null   float32       
 11  date               8766 non-null   datetime64[ns]
 12  range_air_2m       8766 non-null   float32       
 13  range_rhum_2m      8766 non-null   float32       
 14  range_dp

In [5]:
# metrics to monitor during the run

metrics = {
    'true_positives': keras.metrics.TruePositives(name='true_positives'),
    'false_positives': keras.metrics.FalsePositives(name='false_positives'),
    'true_negatives': keras.metrics.TrueNegatives(name='true_negatives'),
    'false_negatives': keras.metrics.FalseNegatives(name='false_negatives'), 
    'AUC': keras.metrics.AUC(name='AUC'),
    'matthews_correlation': data_functions.matthews_correlation,
    'F1': data_functions.f1
}

In [6]:
# set run hyperparameters

hyperparameters = {
    'recurrent_unit_type': 'LSTM',
    'statefullness': False,
    'units': 5,
    'learning_rate': 0.01,
    'batch_size': 128,
    'past_history': 8,
    'dropout': 0.03,
    'class_weight': class_weight,
    'initial_output_bias': output_bias,
    'future_target': 1,
    'step': 1,
    'test_data_frac': 0.25,
    'validation_data_frac': 0.5,
    'training_epochs': 25,
    'verbose': 0
}

# set parameters to vary 

experimental_parameters = ['statefullness', 'RNN_type', 'past_history']

parameter_sets = {
    'Stateless LSTM, past history 4': [False, 'LSTM', 4],
    'Stateless LSTM, past history 8': [False, 'LSTM', 8],
    'Stateless LSTM, past history 16': [False, 'LSTM', 16],
    'Stateless LSTM, past history 32': [False, 'LSTM', 32],
    'Statefull LSTM, past history 4': [True, 'LSTM', 4],
    'Statefull LSTM, past history 8': [True, 'LSTM', 8],
    'Statefull LSTM, past history 16': [True, 'LSTM', 16],
    'Statefull LSTM, past history 32': [True, 'LSTM', 32],
    'Stateless GRU, past history 4': [False, 'GRU', 4],
    'Stateless GRU, past history 8': [False, 'GRU', 8],
    'Stateless GRU, past history 16': [False, 'GRU', 16],
    'Stateless GRU, past history 32': [False, 'GRU', 32],
    'Statefull GRU, past history 4': [True, 'GRU', 4],
    'Statefull GRU, past history 8': [True, 'GRU', 8],
    'Statefull GRU, past history 16': [True, 'GRU', 16],
    'Statefull GRU, past history 32': [True, 'GRU', 32]
}


# set up pandas dataframe to hold results

metric_names = ['loss'] + list(metrics.keys())
metric_mean_names = []
metric_std_names = []

for metric_name in metric_names:
    metric_mean_names.append(metric_name)
    metric_std_names.append(f'{metric_name}_stdev')
    
column_names = ['description', 'peak_memory', 'peak_memory_stdev'] + experimental_parameters + metric_mean_names + metric_std_names
results = pd.DataFrame(columns = column_names)

In [ ]:
# Do training runs on experimental conditions

num_repititions = 3
total_training_runs = num_repititions * len(parameter_sets)

for description, parameter_set in parameter_sets.items():
    run_results = []
    memory_use = []
    
    for i in range(num_repititions):
        print(f'{description}, run {i}, {total_training_runs} remaining')

        LSTM_RNN = NeuralNet(hyperparameters, list(metrics.values()), data)

        LSTM_RNN.statefullness = parameter_set[0]
        LSTM_RNN.recurrent_unit_type = parameter_set[1]
        LSTM_RNN.batch_size = parameter_set[2]

        LSTM_RNN.prep_data()
        
        tracemalloc.start()
        LSTM_RNN.build_model()
        LSTM_RNN.train_model()
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        memory_use.append(peak)

        result = LSTM_RNN.evaluate_model()
        run_results.append(result)
        
        total_training_runs -= 1
        
        del LSTM_RNN.model
        K.clear_session()
        
    run_means = np.array(run_results).mean(axis=0)
    run_stdev = np.array(run_results).std(axis=0)
    peak_memory_mean = np.array(memory_use).mean()
    peak_memory_std = np.array(memory_use).std()
    result_row = [description, peak_memory_mean, peak_memory_std] + parameter_set + list(run_means) + list(run_stdev)
    results.loc[len(results)] = result_row    

Stateless LSTM, past history 4, run 0, 48 remaining
Stateless LSTM, past history 4, run 1, 47 remaining
Stateless LSTM, past history 4, run 2, 46 remaining
Stateless LSTM, past history 8, run 0, 45 remaining
Stateless LSTM, past history 8, run 1, 44 remaining
Stateless LSTM, past history 8, run 2, 43 remaining


In [ ]:
results.head()

In [ ]:
# plot results

results['past_history'] = results['past_history'].astype('float64')
conditions = results.groupby(['statefullness', 'RNN_type'])

plt.subplots(1, 4, figsize=(16,4))

plt.subplot(1, 4, 1)

for name, group in conditions:
    x = np.array(group['past_history'])
    y = np.array(group['matthews_correlation'])
    y_err = np.array(group['matthews_correlation_stdev'])
    plt.plot(x, y, marker="o", linestyle="-", label=name)
    plt.fill_between(x, y-y_err, y+y_err, alpha=0.25)
    
    plt.legend(loc="lower right")
    plt.xlabel('Past history (days)')
    plt.ylabel('Matthews Correlation Coef.')
    
plt.subplot(1, 4, 2)

for name, group in conditions:
    x = np.array(group['past_history'])
    y = np.array(group['AUC'])
    y_err = np.array(group['AUC_stdev'])
    plt.plot(x, y, marker="o", linestyle="-", label=name)
    plt.fill_between(x, y-y_err, y+y_err, alpha=0.25)
    
    plt.legend(loc="lower right")
    plt.xlabel('Past history (days)')
    plt.ylabel('Area under the ROC curve')

plt.subplot(1, 4, 3)
    
for name, group in conditions:
    x = np.array(group['past_history'])
    y = np.array(group['F1'])
    y_err = np.array(group['F1_stdev'])
    plt.plot(x, y, marker="o", linestyle="-", label=name)
    plt.fill_between(x, y-y_err, y+y_err, alpha=0.25)
    
    plt.legend(loc="lower right")
    plt.xlabel('Past history (days)')
    plt.ylabel('F1 score')
    
plt.subplot(1, 4, 4)

for name, group in conditions:
    x = np.array(group['past_history'])
    y = np.array(group['peak_memory'] / 10**6)
    y_err = np.array(group['peak_memory_stdev'] / 10**6)
    plt.plot(x, y, marker="o", linestyle="-", label=name)
    plt.fill_between(x, y-y_err, y+y_err, alpha=0.25)
    
    plt.legend(loc="lower right")
    plt.xlabel('Past history (days)')
    plt.ylabel('Peak memory use (MB)')
    
plt.tight_layout()

In [ ]:
# set run hyperparameters

hyperparameters = {
    'recurrent_unit_type': 'LSTM',
    'statefullness': False,
    'units': 5,
    'learning_rate': 0.01,
    'batch_size': 128,
    'past_history': 32,
    'dropout': 0.03,
    'class_weight': class_weight,
    'initial_output_bias': output_bias,
    'future_target': 1,
    'step': 1,
    'test_data_frac': 0.25,
    'validation_data_frac': 0.5,
    'training_epochs': 50,
    'verbose': 0
}

# set parameters to vary 

experimental_parameters = ['statefullness','batch_size']

parameter_sets = {
    'Stateless GRU, batch size 32': [False, 32],
    'Stateless GRU, batch size 64': [False, 64],
    'Stateless GRU, batch size 128': [False, 128],
    'Stateless GRU, batch size 256': [False, 256],
    'Stateless GRU, batch size 512': [False, 512],
    'Stateless GRU, batch size 1024': [False, 1024],
    'Stateless GRU, batch size 2048': [False, 2048],
    'Statefull GRU, batch size 32': [True, 32],
    'Statefull GRU, batch size 64': [True, 64],
    'Statefull GRU, batch size 128': [True, 128],
    'Statefull GRU, batch size 256': [True, 256],
    'Statefull GRU, batch size 512': [True, 512],
    'Statefull GRU, batch size 1024': [True, 1024],
    'Statefull GRU, batch size 2048': [True, 2084]
}

# set up pandas dataframe to hold results

metric_names = ['loss'] + list(metrics.keys())
metric_mean_names = []
metric_std_names = []

for metric_name in metric_names:
    metric_mean_names.append(metric_name)
    metric_std_names.append(f'{metric_name}_stdev')
    
column_names = ['description', 'peak_memory', 'peak_memory_stdev'] + experimental_parameters + metric_mean_names + metric_std_names
results = pd.DataFrame(columns = column_names)

In [ ]:
# Do training runs on experimental conditions

num_repititions = 3
total_training_runs = num_repititions * len(parameter_sets)

for description, parameter_set in parameter_sets.items():
    run_results = []
    memory_use = []
    
    for i in range(num_repititions):
        print(f'{description}, run {i}, {total_training_runs} remaining')

        GRU_RNN = NeuralNet(hyperparameters, list(metrics.values()), data)

        GRU_RNN.statefullness = parameter_set[0]
        GRU_RNN.past_history = parameter_set[1]

        GRU_RNN.prep_data()
        
        tracemalloc.start()
        GRU_RNN.build_model()
        GRU_RNN.train_model()
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        memory_use.append(peak)

        result = GRU_RNN.evaluate_model()
        run_results.append(result)
        
        total_training_runs -= 1
        
    run_means = np.array(run_results).mean(axis=0)
    run_stdev = np.array(run_results).std(axis=0)
    peak_memory_mean = np.array(memory_use).mean()
    peak_memory_std = np.array(memory_use).std()
    result_row = [description, peak_memory_mean, peak_memory_std] + parameter_set + list(run_means) + list(run_stdev)
    results.loc[len(results)] = result_row   

In [ ]:
# plot results

results['past_history'] = results['past_history'].astype('float64')
conditions = results.groupby(['statefullness'])

plt.subplots(1, 4, figsize=(16,4))

plt.subplot(1, 4, 1)

for name, group in conditions:
    x = np.array(group['past_history'])
    y = np.array(group['matthews_correlation'])
    y_err = np.array(group['matthews_correlation_stdev'])
    plt.plot(x, y, marker="o", linestyle="-", label=name)
    plt.fill_between(x, y-y_err, y+y_err, alpha=0.25)
    
    plt.legend(loc="lower right")
    plt.xlabel('Past history (days)')
    plt.ylabel('Matthews Correlation Coef.')

plt.subplot(1, 4, 2)

for name, group in conditions:
    x = np.array(group['past_history'])
    y = np.array(group['AUC'])
    y_err = np.array(group['AUC_stdev'])
    plt.plot(x, y, marker="o", linestyle="-", label=name)
    plt.fill_between(x, y-y_err, y+y_err, alpha=0.25)
    
    plt.legend(loc="lower right")
    plt.xlabel('Past history (days)')
    plt.ylabel('Area under the ROC curve')

plt.subplot(1, 4, 3)
    
for name, group in conditions:
    x = np.array(group['past_history'])
    y = np.array(group['F1'])
    y_err = np.array(group['F1_stdev'])
    plt.plot(x, y, marker="o", linestyle="-", label=name)
    plt.fill_between(x, y-y_err, y+y_err, alpha=0.25)
    
    plt.legend(loc="lower right")
    plt.xlabel('Past history (days)')
    plt.ylabel('F1 score')
    
plt.subplot(1, 4, 4)

for name, group in conditions:
    x = np.array(group['past_history'])
    y = np.array(group['peak_memory'] / 10**6)
    y_err = np.array(group['peak_memory_stdev'] / 10**6)
    plt.plot(x, y, marker="o", linestyle="-", label=name)
    plt.fill_between(x, y-y_err, y+y_err, alpha=0.25)
    
    plt.legend(loc="lower right")
    plt.xlabel('Past history (days)')
    plt.ylabel('Peak memory use (MB)')
    
plt.tight_layout()

In [ ]:
# set run hyperparameters

hyperparameters = {
    'recurrent_unit_type': 'GRU',
    'statefullness': False,
    'units': 10,
    'learning_rate': 0.01,
    'batch_size': 1024,
    'past_history': 28,
    'dropout': 0.03,
    'class_weight': class_weight,
    'initial_output_bias': output_bias,
    'future_target': 1,
    'step': 1,
    'test_data_frac': 0.25,
    'validation_data_frac': 0.5,
    'training_epochs': 25,
    'verbose': 0
}

# set parameters to vary 

experimental_parameters = ['units']

parameter_sets = {
    'GRU units: 1': [1],
    'GRU units: 2': [2],
    'GRU units: 3': [3],
    'GRU units: 4': [4],
    'GRU units: 5': [5],
    'GRU units: 6': [6],
    'GRU units: 7': [7],
    'GRU units: 8': [8],
    'GRU units: 9': [9],
    'GRU units: 10': [10]
}

# set up pandas dataframe to hold results

metric_names = ['loss'] + list(metrics.keys())
metric_mean_names = []
metric_std_names = []

for metric_name in metric_names:
    metric_mean_names.append(metric_name)
    metric_std_names.append(f'{metric_name}_stdev')
    
column_names = ['description', 'peak_memory', 'peak_memory_stdev'] + experimental_parameters + metric_mean_names + metric_std_names
results = pd.DataFrame(columns = column_names)

In [ ]:
# Do training runs on experimental conditions

num_repititions = 10
total_training_runs = num_repititions * len(parameter_sets)

for description, parameter_set in parameter_sets.items():
    run_results = []
    memory_use = []
    
    for i in range(num_repititions):
        print(f'{description}, run {i}, {total_training_runs} remaining')

        GRU_RNN = NeuralNet(hyperparameters, list(metrics.values()), data)

        GRU_RNN.units = parameter_set[0]

        GRU_RNN.prep_data()
        
        tracemalloc.start()
        GRU_RNN.build_model()
        GRU_RNN.train_model()
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        memory_use.append(peak)

        result = GRU_RNN.evaluate_model()
        run_results.append(result)
        
        total_training_runs -= 1
        
    run_means = np.array(run_results).mean(axis=0)
    run_stdev = np.array(run_results).std(axis=0)
    peak_memory_mean = np.array(memory_use).mean()
    peak_memory_std = np.array(memory_use).std()
    result_row = [description, peak_memory_mean, peak_memory_std] + parameter_set + list(run_means) + list(run_stdev)
    results.loc[len(results)] = result_row 

In [ ]:
# plot results

results['units'] = results['units'].astype('float64')

plt.subplots(1, 4, figsize=(16,4))

plt.subplot(1, 4, 1)

x = np.array(results['units'])
y = np.array(results['matthews_correlation'])
y_err = np.array(results['matthews_correlation_stdev'])
plt.plot(x, y, marker="o", linestyle="-", label=name)
plt.fill_between(x, y-y_err, y+y_err, alpha=0.25)

plt.legend(loc="lower right")
plt.xlabel('GRU units')
plt.ylabel('Matthews Correlation Coef.')
    
plt.subplot(1, 4, 2)

x = np.array(results['units'])
y = np.array(results['AUC'])
y_err = np.array(results['AUC_stdev'])
plt.plot(x, y, marker="o", linestyle="-", label=name)
plt.fill_between(x, y-y_err, y+y_err, alpha=0.25)

plt.legend(loc="lower right")
plt.xlabel('GRU units')
plt.ylabel('Area under the ROC curve')

plt.subplot(1, 4, 3)
    
x = np.array(results['units'])
y = np.array(results['F1'])
y_err = np.array(results['F1_stdev'])
plt.plot(x, y, marker="o", linestyle="-", label=name)
plt.fill_between(x, y-y_err, y+y_err, alpha=0.25)

plt.legend(loc="lower right")
plt.xlabel('GRU units')
plt.ylabel('F1 score')
    
plt.subplot(1, 4, 4)

x = np.array(results['units'])
y = np.array(results['peak_memory'] / 10**6)
y_err = np.array(results['peak_memory_stdev'] / 10**6)
plt.plot(x, y, marker="o", linestyle="-", label=name)
plt.fill_between(x, y-y_err, y+y_err, alpha=0.25)

plt.legend(loc="lower right")
plt.xlabel('GRU units')
plt.ylabel('Peak memory use (MB)')
    
plt.tight_layout()

In [ ]:
# set run hyperparameters

hyperparameters = {
    'recurrent_unit_type': 'GRU',
    'statefullness': False,
    'units': 10,
    'learning_rate': 0.01,
    'batch_size': 1024,
    'past_history': 28,
    'dropout': 0.01,
    'class_weight': class_weight,
    'initial_output_bias': output_bias,
    'future_target': 1,
    'step': 1,
    'test_data_frac': 0.25,
    'validation_data_frac': 0.5,
    'training_epochs': 25,
    'verbose': 0
}

# set parameters to vary 

experimental_parameters = ['dropout']

parameter_sets = {
    'Dropout: 0.01': [0.01],
    'Dropout: 0.05': [0.05],
    'Dropout: 0.10': [0.10],
    'Dropout: 0.15': [0.15],
    'Dropout: 0.20': [0.20],
    'Dropout: 0.25': [0.25],
    'Dropout: 0.30': [0.30]
}

# set up pandas dataframe to hold results

metric_names = ['loss'] + list(metrics.keys())
metric_mean_names = []
metric_std_names = []

for metric_name in metric_names:
    metric_mean_names.append(metric_name)
    metric_std_names.append(f'{metric_name}_stdev')
    
column_names = ['description', 'peak_memory', 'peak_memory_stdev'] + experimental_parameters + metric_mean_names + metric_std_names
results = pd.DataFrame(columns = column_names)

In [ ]:
# Do training runs on experimental conditions

num_repititions = 20
total_training_runs = num_repititions * len(parameter_sets)

for description, parameter_set in parameter_sets.items():
    run_results = []
    memory_use = []
    
    for i in range(num_repititions):
        print(f'{description}, run {i}, {total_training_runs} remaining')

        GRU_RNN = NeuralNet(hyperparameters, list(metrics.values()), data)
        GRU_RNN.dropout = parameter_set[0]
        GRU_RNN.prep_data()
        
        tracemalloc.start()
        GRU_RNN.build_model()
        GRU_RNN.train_model()
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        memory_use.append(peak)

        result = GRU_RNN.evaluate_model()
        run_results.append(result)
        
        total_training_runs -= 1
        
    run_means = np.array(run_results).mean(axis=0)
    run_stdev = np.array(run_results).std(axis=0)
    peak_memory_mean = np.array(memory_use).mean()
    peak_memory_std = np.array(memory_use).std()
    result_row = [description, peak_memory_mean, peak_memory_std] + parameter_set + list(run_means) + list(run_stdev)
    results.loc[len(results)] = result_row 

In [ ]:
# plot results

#results['units'] = results['units'].astype('float64')

plt.subplots(1, 4, figsize=(16,4))

plt.subplot(1, 4, 1)

x = np.array(results['dropout'])
y = np.array(results['matthews_correlation'])
y_err = np.array(results['matthews_correlation_stdev'])
plt.plot(x, y, marker="o", linestyle="-", label=name)
plt.fill_between(x, y-y_err, y+y_err, alpha=0.25)

plt.legend(loc="lower right")
plt.xlabel('Dropout rate')
plt.ylabel('Matthews Correlation Coef.')
    
plt.subplot(1, 4, 2)

x = np.array(results['dropout'])
y = np.array(results['AUC'])
y_err = np.array(results['AUC_stdev'])
plt.plot(x, y, marker="o", linestyle="-", label=name)
plt.fill_between(x, y-y_err, y+y_err, alpha=0.25)

plt.legend(loc="lower right")
plt.xlabel('Dropout rate')
plt.ylabel('Area under the ROC curve')

plt.subplot(1, 4, 3)
    
x = np.array(results['dropout'])
y = np.array(results['F1'])
y_err = np.array(results['F1_stdev'])
plt.plot(x, y, marker="o", linestyle="-", label=name)
plt.fill_between(x, y-y_err, y+y_err, alpha=0.25)

plt.legend(loc="lower right")
plt.xlabel('Dropout rate')
plt.ylabel('F1 score')
    
plt.subplot(1, 4, 4)

x = np.array(results['dropout'])
y = np.array(results['peak_memory'] / 10**6)
y_err = np.array(results['peak_memory_stdev'] / 10**6)
plt.plot(x, y, marker="o", linestyle="-", label=name)
plt.fill_between(x, y-y_err, y+y_err, alpha=0.25)

plt.legend(loc="lower right")
plt.xlabel('Dropout rate')
plt.ylabel('Peak memory use (MB)')
    
plt.tight_layout()